## Forecast next 6 months of revenue for a selected restaurant

In [8]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing


df = pd.read_excel('Cleaned_Seasonality_Data.xlsx')


df['Month-Year'] = df['Transaction Date'].dt.to_period('M').dt.to_timestamp()


restaurants = df['Restaurant Name'].unique()


forecast_records = []


for restaurant in restaurants:
    venue_df = df[df['Restaurant Name'] == restaurant]

    
    monthly = venue_df.groupby('Month-Year')['Revenue'].sum().sort_index()

   
    if len(monthly) < 3:
        continue

    
    if len(monthly) >= 24:
        seasonal_periods = 12
    elif len(monthly) >= 12:
        seasonal_periods = 6
    else:
        seasonal_periods = None

    try:
       
        if seasonal_periods:
            model = ExponentialSmoothing(
                monthly,
                trend='add',
                seasonal='add',
                seasonal_periods=seasonal_periods
            ).fit()
        else:
            model = ExponentialSmoothing(
                monthly,
                trend='add',
                seasonal=None
            ).fit()

     
        forecast = model.forecast(6)

        
        for date, value in forecast.items():
            forecast_records.append({
                'Restaurant Name': restaurant,
                'Forecast Month': date.strftime('%Y-%m'),
                'Forecasted Revenue': round(max(value, 0), 2)
            })

    except Exception as e:
       
        continue 


forecast_df = pd.DataFrame(forecast_records)
forecast_df.to_excel('6_Month_Revenue_Forecast.xlsx', index=False)

print("Forecast saved to '6_Month_Revenue_Forecast.xlsx'")


Forecast saved to '6_Month_Revenue_Forecast.xlsx'
